
## Home Task 


The Task is to extract relevant infromation from the messy medical data using regex.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

1) Extract the date strings
Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

2) Normalize the extracted dates considering the following rules:  
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

3) Sort records in ascending chronological order accoring.




### Get data

In [159]:
import numpy as np
import pandas as pd
import re

In [160]:
def get_data():
    with open('dates.txt', 'r') as file:
        data = file.read()
    
    return data
    
data = get_data()
print(data[:500])

03/25/93 Total time of visit (in minutes):
6/18/85 Primary Care Doctor:
sshe plans to move as of 7/8/71 In-Home Services: None
7 on 9/27/75 Audit C Score Current:
2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7
.Per 7/06/79 Movement D/O note:
4, 5/18/78 Patient's thoughts about current substance abuse:
10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview
3/7/86 SOS-10 Total Score:
(4/10/71)Score-1Audit C Score Current:
(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activ


### Extract the date strings

In [161]:
def extract_date(data):
    # Define the date patterns
    date_patterns = [
        r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',  # 08/11/1953, 8/11/53, 8/11/1953, 8/3/53
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* \d{1,2}[-,]* \d{4}',  # Jan-11-1953; Jan 11, 1953; January 11, 1953
        r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.,]* \d{4}',  # 11 Feb 1953; 11 March 1953; 11 Mar. 1953
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}[a-z]*, \d{4}',  # Mar 11th, 1953; Mar 11st, 1953; Mar 11nd, 1953
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}',  # Feb 1953; Sep 1953; Oct 1953
        r'\d{1,2}/\d{4}',  # 6/1953, 12/1953
        r'[12]\d{3}'  # 1953, 1954
    ]

    
    date_pattern = '|'.join(date_patterns)

    extracted_data = re.findall(date_pattern, data)
        
    return extracted_data


extracted_data = extract_date(data)
print("Len: ",len(extracted_data))
extracted_data

Len:  501


['03/25/93',
 '6/18/85',
 '7/8/71',
 '9/27/75',
 '2/6/96',
 '7/06/79',
 '5/18/78',
 '10/24/89',
 '3/7/86',
 '4/10/71',
 '5/11/85',
 '4/09/75',
 '8/01/98',
 '1/26/72',
 '5/24/1990',
 '1/25/2011',
 '4/12/82',
 '10/13/1976',
 '4/24/98',
 '5/21/77',
 '7/21/98',
 '10/21/79',
 '3/03/90',
 '2/11/76',
 '07/25/1984',
 '4-13-82',
 '9/22/89',
 '9/02/76',
 '9/12/71',
 '10/24/86',
 '03/31/1985',
 '7/20/72',
 '4/12/87',
 '06/20/91',
 '5/12/2012',
 '3/15/83',
 '2/14/73',
 '5/24/88',
 '7/27/1986',
 '1-14-81',
 '7-29-75',
 '6/24/87',
 '8/14/94',
 '4/13/2002',
 '8/16/82',
 '2/15/1998',
 '7/15/91',
 '06/12/94',
 '9/17/84',
 '2/28/75',
 '11/22/75',
 '5/24/91',
 '6/13/92',
 '7/11/71',
 '12/26/86',
 '10/11/1987',
 '3/14/95',
 '12/01/73',
 '12/5/2010',
 '08/20/1982',
 '7/24/95',
 '8/06/83',
 '02/22/92',
 '6/28/87',
 '07/29/1994',
 '08/11/78',
 '10/29/91',
 '7/6/91',
 '1/21/87',
 '11/3/1985',
 '7/04/82',
 '4-13-89',
 '7/11/77',
 '9/36/308',
 '4/12/74',
 '09/19/81',
 '9/6/79',
 '12/5/87',
 '01/05/1999',
 '4/22

### Normalize the extracted dates

In [162]:
def get_year_month_day(match):
        year = int(match.group('year'))
        month = str(match.group('month')) if match.group('month') else 1
        day = int(match.group('day')) if match.group('day') else 1
        return year, month, day

def normalize(data_ext):
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
              "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    normalized_data = []
    
    
    patterns = [
        r'(?:(?P<month>\d{1,2})[ -/.,])?(?:(?P<day>\d{1,2})[ -/.,])?(?P<year>\d{2,4})$',
        r'(?:(?P<day>\d{1,2})[ /.,-])?(?P<month>[a-zA-Z]+)[ /.,-]?(?P<year>\d{4})$',
        r'(?:(?P<month>[a-zA-Z]+)[\s.]+)?(?:(?P<day>\d{1,2})[ /.,-]+)?(?P<year>\d{4})$',
        r'^(?P<month>[A-Za-z]+)\s+(?P<year>\d+)$'
    ]
    
    for data in data_ext:
        match = None
        for pattern in patterns:
            match = re.match(pattern, data)
            if match:
                break

        if match:
            year, month, day = get_year_month_day(match)

        check = True
        
        if year < 1000:
            if year < 100:
                year += 1900
            else:
                check = False
    
        if check == True:
            if isinstance(month, str):
                if len(month) >= 3:
                    month = month[:3]
                    month = months.index(month) + 1
                month = int(month)

            normalized_data_date = "{:04d}-{:02d}-{:02d}".format(year, month, day)
            normalized_data.append(normalized_data_date)
    
    normalized_data = pd.DataFrame({"res": normalized_data, "index": range(0, len(normalized_data))})

    return normalized_data

normalized_data = normalize(extracted_data)
normalized_data


,res,index
0,1993-03-25,0
1,1985-06-18,1
2,1971-07-08,2
3,1975-09-27,3
4,1996-02-06,4
...,...,...
495,1979-01-01,495
496,2006-01-01,496
497,2008-01-01,497
498,2005-01-01,498


### Sort records in ascending chronological order

In [164]:
normalized_data = normalized_data.sort_values(by='res')
normalized_data = normalized_data.reset_index(drop=True)

normalized_data

,res,index
0,1971-04-10,9
1,1971-05-18,84
2,1971-07-08,2
3,1971-07-11,53
4,1971-09-12,28
...,...,...
495,2016-05-01,231
496,2016-05-30,141
497,2016-10-13,186
498,2016-10-19,161


<font color = blue >

### Expected Output

</font>

<table align= 'left'>
    <tr><td></td><td> res</td><td>index</td></tr>
    <tr><td>0</td><td> 1971-04-10</td><td>9</td></tr>    
    <tr><td>1</td><td>1971-05-18</td><td>84</td></tr>    
    <tr><td>2</td><td>1971-07-08</td><td>2</td></tr>    
    <tr><td>3</td><td>1971-07-11</td><td>53</td></tr>    
    <tr><td>4</td><td>1971-09-12</td><td>28</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>495 </td><td>2016-05-01</td><td>    427</td></tr>    
    <tr><td>496</td><td> 2016-05-30  </td><td>  141</td></tr>    
    <tr><td>497</td><td> 2016-10-13  </td><td>  186</td></tr>    
    <tr><td>498</td><td> 2016-10-19   </td><td> 161</td></tr>    
    <tr><td>499 </td><td>2016-11-01 </td><td>   413</td></tr>    
</table> 